## **INSTALAÇÃO DE REQUERIMENTOS**

In [ ]:
!git clone https://github.com/saulocatharino/dust3r.git

Cloning into 'dust3r'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 154 (delta 40), reused 29 (delta 29), pack-reused 89
Receiving objects: 100% (154/154), 587.68 KiB | 2.02 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [ ]:
%cd /content/dust3r

/content/dust3r


In [ ]:
!git submodule update --init --recursive

Submodule 'croco' (https://github.com/naver/croco) registered for path 'croco'
Cloning into '/content/dust3r/croco'...
Submodule path 'croco': checked out '743ee71a2a9bf57cea6832a9064a70a0597fcfcb'


In [ ]:
!mkdir -p checkpoints/
!wget https://download.europe.naverlabs.com/ComputerVision/DUSt3R/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth -P checkpoints/

--2024-04-16 23:34:04--  https://download.europe.naverlabs.com/ComputerVision/DUSt3R/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
Resolving download.europe.naverlabs.com (download.europe.naverlabs.com)... 110.234.56.25
Connecting to download.europe.naverlabs.com (download.europe.naverlabs.com)|110.234.56.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2285019929 (2.1G)
Saving to: ‘checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth’

DUSt3R_ViTLarge_Bas 100%[===================>]   2.13G  19.0MB/s    in 1m 59s  

2024-04-16 23:36:04 (18.3 MB/s) - ‘checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth’ saved [2285019929/2285019929]



In [ ]:
# DUST3R relies on RoPE positional embeddings for which you can compile some cuda kernels for faster runtime.
%cd croco/models/curope/
!python setup.py build_ext --inplace
%cd ../../../

/content/dust3r/croco/models/curope
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:500: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:415: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:425: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.2
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
building 'curope' extension
creating build
creating build/temp.linu

In [ ]:
!pip install -r requirements_optional.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 50.5 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.


In [ ]:
%cd /content/dust3r

/content/dust3r


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.8/693.8 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_6

## **EXPORTA MODELO 3D**

In [ ]:
%cd /content/dust3r
import os
import torch
import numpy as np
import trimesh
import copy
from scipy.spatial.transform import Rotation
from dust3r.inference import inference, load_model
from dust3r.image_pairs import make_pairs
from dust3r.utils.image import load_images, rgb
from dust3r.utils.device import to_numpy
from dust3r.viz import add_scene_cam, CAM_COLORS, OPENGL, pts3d_to_trimesh, cat_meshes
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

def convert_scene_output_to_glb(outdir, imgs, pts3d, mask, focals, cams2world, output_file, cam_size=0.05,
                                cam_color=None, as_pointcloud=False,
                                transparent_cams=False, silent=False):
    assert len(pts3d) == len(mask) <= len(imgs) <= len(cams2world) == len(focals)
    pts3d = to_numpy(pts3d)
    imgs = to_numpy(imgs)
    focals = to_numpy(focals)
    cams2world = to_numpy(cams2world)

    scene = trimesh.Scene()

    # Full pointcloud or mesh
    if as_pointcloud:
        pts = np.concatenate([p[m] for p, m in zip(pts3d, mask)])
        col = np.concatenate([p[m] for p, m in zip(imgs, mask)])
        pct = trimesh.PointCloud(pts.reshape(-1, 3), colors=col.reshape(-1, 3))
        scene.add_geometry(pct)
    else:
        meshes = []
        for i in range(len(imgs)):
            meshes.append(pts3d_to_trimesh(imgs[i], pts3d[i], mask[i]))
        mesh = trimesh.Trimesh(**cat_meshes(meshes))
        scene.add_geometry(mesh)

    # Add each camera
    for i, pose_c2w in enumerate(cams2world):
        camera_edge_color = cam_color[i % len(cam_color)] if isinstance(cam_color, list) else cam_color or CAM_COLORS[i % len(CAM_COLORS)]
        add_scene_cam(scene, pose_c2w, camera_edge_color,
                      None if transparent_cams else imgs[i], focals[i],
                      imsize=imgs[i].shape[1::-1], screen_width=cam_size)

    # Apply transformation and export GLB file
    rot = np.eye(4)
    rot[:3, :3] = Rotation.from_euler('y', np.deg2rad(180)).as_matrix()
    scene.apply_transform(np.linalg.inv(cams2world[0] @ OPENGL @ rot))
    outfile = os.path.join(outdir, output_file)
    if not silent:
        print('(exporting 3D scene to', outfile, ')')
    scene.export(file_obj=outfile)
    return outfile

def get_3D_model_from_scene(outdir, silent, scene, output_file, min_conf_thr=3, as_pointcloud=False, mask_sky=True,
                            clean_depth=True, transparent_cams=True, cam_size=0.00):
    if scene is None:
        return None

    # Post-processing
    if clean_depth:
        scene = scene.clean_pointcloud()
    if mask_sky:
        scene = scene.mask_sky()

    # Get optimized values from scene
    rgbimg = scene.imgs
    focals = scene.get_focals().cpu()
    cams2world = scene.get_im_poses().cpu()
    pts3d = to_numpy(scene.get_pts3d())
    scene.min_conf_thr = float(scene.conf_trf(torch.tensor(min_conf_thr)))
    msk = to_numpy(scene.get_masks())
    return convert_scene_output_to_glb(outdir, rgbimg, pts3d, msk, focals, cams2world, output_file, as_pointcloud=as_pointcloud,
                                       transparent_cams=transparent_cams, cam_size=cam_size, silent=silent)

def main(weights_path, image_folder, output_file):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = load_model(weights_path, device=device)

    imgs = load_images(image_folder, size=512)
    if len(imgs) == 1:
        imgs = [imgs[0], copy.deepcopy(imgs[0])]
        imgs[1]['idx'] = 1

    tam = len(imgs)-1

    scenegraph_type = f"swin-4"
    #scenegraph_type = f"oneref-{tam}"

    pairs = make_pairs(imgs, scene_graph=scenegraph_type, prefilter=None, symmetrize=True)
    output = inference(pairs, model, device)

    mode = GlobalAlignerMode.PointCloudOptimizer if len(imgs) > 2 else GlobalAlignerMode.PairViewer
    scene = global_aligner(output, device=device, mode=mode)

    if mode == GlobalAlignerMode.PointCloudOptimizer:
        loss = scene.compute_global_alignment(init='mst')

    outfile = get_3D_model_from_scene('', False, scene, output_file)

if __name__ == '__main__':
    weights_path = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
    image_folder = "sala"
    output_file = "sala.glb"
    main(weights_path, image_folder, output_file)


/content/dust3r
... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>
>> Loading images from sala
 - adding IMG-20240416-WA0007.jpg with resolution 1600x1200 --> 512x384
 - adding IMG-20240416-WA0008.jpg with resolution 1600x1200 --> 512x384
 - adding IMG-20240416-WA0009.jpg with resolution 1600x1200 --> 512x384
 - adding IMG-20240416-WA0010.jpg with resolution 1600x1200 --> 512x384
 - adding IMG-20240416-WA0012.jpg with resolution 1600x1200 --> 512x384
 - adding IMG-20240416-WA0013.jpg with resolution 1600x1200 --> 512x384
 - adding IMG-20240416-WA0016.jpg with resolution 1600x1200 --> 512x384
 - add

100%|██████████| 26/26 [02:18<00:00,  5.33s/it]


 init edge (13*,14*) score=189.3213653564453
 init edge (13,10*) score=77.46690368652344
 init edge (14,16*) score=58.00434875488281
 init edge (19*,16) score=10.265995025634766
 init edge (15*,13) score=6.484203338623047
 init edge (20*,19) score=5.9479756355285645
 init edge (10,7*) score=2.7390027046203613
 init edge (20,18*) score=2.4757490158081055
 init edge (19,21*) score=132.26380920410156
 init edge (22*,21) score=46.69668197631836
 init edge (7,6*) score=24.63801383972168
 init edge (22,24*) score=20.064966201782227
 init edge (7,4*) score=15.021427154541016
 init edge (23*,22) score=9.625177383422852
 init edge (20,17*) score=7.667440891265869
 init edge (8*,6) score=37.21819305419922
 init edge (25*,23) score=18.31930923461914
 init edge (8,5*) score=13.921257019042969
 init edge (11*,8) score=3.011971950531006
 init edge (25,0*) score=32.65842056274414
 init edge (3*,0) score=29.032541275024414
 init edge (2*,3) score=26.897125244140625
 init edge (9*,11) score=9.631421089

100%|██████████| 300/300 [02:06<00:00,  2.37it/s, lr=1.27413e-06 loss=0.00361034]


(exporting 3D scene to sala.glb )


## **EXPORTAR DATASET**

In [ ]:
%cd /content/dust3r
import os
import torch
import numpy as np
import trimesh
import copy
from scipy.spatial.transform import Rotation
from dust3r.inference import inference, load_model
from dust3r.image_pairs import make_pairs
from dust3r.utils.image import load_images, rgb
from dust3r.utils.device import to_numpy
from dust3r.viz import add_scene_cam, CAM_COLORS, OPENGL, pts3d_to_trimesh, cat_meshes
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
import cv2
from google.colab.patches import cv2_imshow
import glob

def convert_scene_output_to_glb(outdir, imgs, pts3d, mask, focals, cams2world, output_file, cam_size=0.05,
                                cam_color=None, as_pointcloud=False,
                                transparent_cams=False, silent=False):
    assert len(pts3d) == len(mask) <= len(imgs) <= len(cams2world) == len(focals)
    pts3d = to_numpy(pts3d)
    imgs = to_numpy(imgs)
    focals = to_numpy(focals)
    cams2world = to_numpy(cams2world)

    scene = trimesh.Scene()

    # Full pointcloud or mesh
    if as_pointcloud:
        pts = np.concatenate([p[m] for p, m in zip(pts3d, mask)])
        col = np.concatenate([p[m] for p, m in zip(imgs, mask)])
        pct = trimesh.PointCloud(pts.reshape(-1, 3), colors=col.reshape(-1, 3))
        scene.add_geometry(pct)
    else:
        meshes = []
        for i in range(len(imgs)):
            meshes.append(pts3d_to_trimesh(imgs[i], pts3d[i], mask[i]))
        mesh = trimesh.Trimesh(**cat_meshes(meshes))
        scene.add_geometry(mesh)

    # Add each camera
    for i, pose_c2w in enumerate(cams2world):
        camera_edge_color = cam_color[i % len(cam_color)] if isinstance(cam_color, list) else cam_color or CAM_COLORS[i % len(CAM_COLORS)]
        add_scene_cam(scene, pose_c2w, camera_edge_color,
                      None if transparent_cams else imgs[i], focals[i],
                      imsize=imgs[i].shape[1::-1], screen_width=cam_size)

    # Apply transformation and export GLB file
    rot = np.eye(4)
    rot[:3, :3] = Rotation.from_euler('y', np.deg2rad(180)).as_matrix()
    scene.apply_transform(np.linalg.inv(cams2world[0] @ OPENGL @ rot))
    outfile = os.path.join(outdir, output_file)
    if not silent:
        print('(exporting 3D scene to', outfile, ')')
    scene.export(file_obj=outfile)
    return outfile

def get_3D_model_from_scene(outdir, silent, scene, output_file, min_conf_thr=3, as_pointcloud=False, mask_sky=True,
                            clean_depth=True, transparent_cams=True, cam_size=0.00):
    if scene is None:
        return None

    # Post-processing
    if clean_depth:
        scene = scene.clean_pointcloud()
    if mask_sky:
        scene = scene.mask_sky()

    # Get optimized values from scene
    rgbimg = scene.imgs
    focals = scene.get_focals().cpu()
    cams2world = scene.get_im_poses().cpu()
    pts3d = to_numpy(scene.get_pts3d())
    scene.min_conf_thr = float(scene.conf_trf(torch.tensor(min_conf_thr)))
    msk = to_numpy(scene.get_masks())
    return convert_scene_output_to_glb(outdir, rgbimg, pts3d, msk, focals, cams2world, output_file, as_pointcloud=as_pointcloud,
                                       transparent_cams=transparent_cams, cam_size=cam_size, silent=silent)

def main(weights_path, image_folder, output_folder, output_file="modelo.glb"):
    i = 1
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = load_model(weights_path, device=device)

    im_fold_tem = glob.glob(f"{image_folder}/*.png")
    for im in im_fold_tem:
      os.system(f"cp {im} temp_img/1.png")
      imgs = load_images('temp_img', size=512)
      if len(imgs) == 1:
          imgs = [imgs[0], copy.deepcopy(imgs[0])]
          imgs[1]['idx'] = 1

      print(imgs[0].keys())
      tam = len(imgs)-1

      scenegraph_type = f"swin-{tam}"
      #scenegraph_type = f"oneref-{tam}"

      pairs = make_pairs(imgs, scene_graph=scenegraph_type, prefilter=None, symmetrize=True)
      output = inference(pairs, model, device)

      mode = GlobalAlignerMode.PointCloudOptimizer if len(imgs) > 2 else GlobalAlignerMode.PairViewer
      scene = global_aligner(output, device=device, mode=mode)

      if mode == GlobalAlignerMode.PointCloudOptimizer:
          loss = scene.compute_global_alignment(init='mst')

      pts3d = to_numpy(scene.get_pts3d())

      for pts in pts3d:
        normalizedpts3d = cv2.normalize(pts,  pts, 0, 255, cv2.NORM_MINMAX)
        one_channel_3d = cv2.cvtColor(normalizedpts3d, cv2.COLOR_BGR2GRAY)
        cv2_imshow(one_channel_3d)
        cv2.imwrite(f"{output_folder}/{i}.jpg", one_channel_3d)
        print(f"Imagem convertida e salva em {output_folder}/{i}.jpg")
        i += 1

if __name__ == '__main__':
    weights_path = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
    image_folder = "dataset_top"
    output_folder = 'output3d_top'
    main(weights_path, image_folder, output_folder)


In [ ]:
%cd /content/dust3r
!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8s-seg.pt

In [ ]:
!pip install ultralytics==8.0.28

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/dust3r

In [ ]:
!cp /content/drive/MyDrive/engord.ai/estimativa_de_peso_vivo_03.keras ./
!cp /content/drive/MyDrive/engord.ai/dataset.csv ./


## **INFERÊNCIA PESO GADO (sem streamlit)**

In [ ]:
%cd /content/dust3r
import torch
import numpy as np
import copy
from dust3r.inference import inference, load_model
from dust3r.image_pairs import make_pairs
from dust3r.utils.image import load_images
from dust3r.utils.device import to_numpy
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
import pandas as pd

weights_path = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def create_model():
  campos= ['live weithg', 'withers height', 'hip height', 'chest depth', 'chest width', 'ilium width', 'hip joint width', 'oblique body length', 'hip length', 'heart girth']

  dim = 256
  input_shape = (dim, dim, 1)

  # Crie duas entradas separadas
  depth_input_1 = Input(shape=input_shape, name='depth_input_1')
  depth_input_2 = Input(shape=input_shape, name='depth_input_2')

  # Continue com a arquitetura da sua rede
  x1 = Conv2D(64, (3, 3), activation='relu')(depth_input_1)
  x1 = MaxPooling2D((2, 2))(x1)
  x1 = Conv2D(128, (3, 3), activation='relu')(x1)
  x1 = MaxPooling2D((2, 2))(x1)
  x1 = Conv2D(256, (3, 3), activation='relu')(x1)
  x1 = MaxPooling2D((2, 2))(x1)
  x1 = Flatten()(x1)

  x2 = Conv2D(64, (3, 3), activation='relu')(depth_input_2)
  x2 = MaxPooling2D((2, 2))(x2)
  x2 = Conv2D(128, (3, 3), activation='relu')(x2)
  x2 = MaxPooling2D((2, 2))(x2)
  x2 = Conv2D(256, (3, 3), activation='relu')(x2)
  x2 = MaxPooling2D((2, 2))(x2)
  x2 = Flatten()(x2)

  # Concatene as saídas das camadas convolucionais
  concatenated_output = Concatenate()([x1, x2])

  x = Dense(256, activation='relu')(concatenated_output)

  # Saída
  output = Dense(len(campos), activation='linear')(x)

  # Construa o modelo com as duas entradas
  model = Model(inputs=[depth_input_1, depth_input_2], outputs=output)

  # Compile o modelo
  model.compile(optimizer='adam', loss='mean_squared_error')
  return model

inference_model = create_model()
inference_model.load_weights("estimativa_de_peso_vivo_03.keras")
model = load_model(weights_path, device=device)
model_seg = YOLO('yolov8s-seg.pt')

# Carregando o CSV com os dados
data = pd.read_csv('dataset.csv')

# Dicionário de tradução das saídas
traducao = {
    'live weithg': 'Peso Vivo',
    'withers height': 'Altura da Cernelha',
    'hip height': 'Altura do Quadril',
    'chest depth': 'Profundidade do Peito',
    'chest width': 'Largura do Peito',
    'ilium width': 'Largura do Ílio',
    'hip joint width': 'Largura da Articulação do Quadril',
    'oblique body length': 'Comprimento do Corpo Oblíquo',
    'hip length': 'Comprimento do Quadril',
    'heart girth': 'Circunferência Torácica'
}


def get_depth(image):
    depths = []
    i = 1
    cv2.imwrite('/content/dust3r/temp_img/1.png',image)
    imgs = load_images('/content/dust3r/temp_img', size=512)
    if len(imgs) == 1:
        imgs = [imgs[0], copy.deepcopy(imgs[0])]
        imgs[1]['idx'] = 1

    scenegraph_type = f"swin-1"

    pairs = make_pairs(imgs, scene_graph=scenegraph_type, prefilter=None, symmetrize=True)
    output = inference(pairs, model, device)

    mode = GlobalAlignerMode.PointCloudOptimizer if len(imgs) > 2 else GlobalAlignerMode.PairViewer
    scene = global_aligner(output, device=device, mode=mode)

    if mode == GlobalAlignerMode.PointCloudOptimizer:
        loss = scene.compute_global_alignment(init='mst')

    pts3d = to_numpy(scene.get_pts3d())

    for pts in pts3d:
      normalizedpts3d = cv2.normalize(pts,  pts, 0, 255, cv2.NORM_MINMAX)
      one_channel_3d = cv2.cvtColor(normalizedpts3d, cv2.COLOR_BGR2GRAY)
      one_channel_3d = cv2.resize(one_channel_3d, (256,256))
      depths.append(one_channel_3d)
      print(f"Imagem convertida {i}")
      i += 1
    return depths

def get_mask(image):
  results = model_seg.predict(image.copy(), conf=0.40)
  masks = results[0].masks
  mask1 = masks.masks
  mask = mask1.data.cpu().numpy()
  mask = mask.squeeze()
  return mask



dim = 256
name_file = '20.png'
image = cv2.imread(name_file)


fig0, ax0 = plt.subplots()
ax0.imshow(image)

depths = get_depth(image)
depth = depths[0]
fig1, ax1 = plt.subplots()
ax1.imshow(depth)



mask = get_mask(image)


resize_depth = cv2.resize(depth,(dim,dim))
norm_depth = resize_depth / 255.

depth_fim = np.expand_dims(norm_depth, axis=0)
mask_fim = cv2.resize(mask,(256,256))
fig2, ax2 = plt.subplots()
ax2.imshow(mask_fim)

plt.show()
mask_fim = np.expand_dims(mask_fim, axis=0)
prediction = inference_model.predict([depth_fim, mask_fim])


predictions = []
try:
    print("---------------------")
    print("PREDIÇÕES")
    for i, campo in enumerate(traducao.keys()):
        print(f'{traducao[campo]}: {round(float(prediction[0][i]),2)}')
        predictions.append(prediction[0][i])

except:
    pass

N = int(name_file.split(".")[0])
precisao_total = []
print("---------------------")
print("DADOS REAIS")
for i,campo in enumerate(traducao.keys()):
    if campo == "N" or 'time' in campo or 'Data' in campo:
        continue
    v = float(data.iloc[N][campo])
    p = predictions[i]
    per = v/ 100
    diff = abs(v-p)
    percent = 100 - (diff/per)
    precisao_total.append(percent)
    print(f'{traducao[campo]}: {data.iloc[N][campo]}', f' (Precisão: {round(percent,2)}%)')

precisao_modelo = sum(precisao_total)/len(precisao_total)
print('-----------------')
print(f"Precisão do Modelo: {round(precisao_modelo,2)}%")

## **INFERENCIA PESO GADO (com streamlit)**


In [ ]:
!pip install streamlit

In [ ]:
!npm install localtunnel

In [ ]:
%cd /content/dust3r

In [ ]:
%%writefile app.py
import streamlit as st
import torch
import numpy as np
import copy
from dust3r.inference import inference, load_model
from dust3r.image_pairs import make_pairs
from dust3r.utils.image import load_images
from dust3r.utils.device import to_numpy
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
import pandas as pd

st.set_page_config(layout="wide")

weights_path = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def create_model():
  campos= ['live weithg', 'withers height', 'hip height', 'chest depth', 'chest width', 'ilium width', 'hip joint width', 'oblique body length', 'hip length', 'heart girth']

  dim = 256
  input_shape = (dim, dim, 1)

  # Crie duas entradas separadas
  depth_input_1 = Input(shape=input_shape, name='depth_input_1')
  depth_input_2 = Input(shape=input_shape, name='depth_input_2')

  # Continue com a arquitetura da sua rede
  x1 = Conv2D(64, (3, 3), activation='relu')(depth_input_1)
  x1 = MaxPooling2D((2, 2))(x1)
  x1 = Conv2D(128, (3, 3), activation='relu')(x1)
  x1 = MaxPooling2D((2, 2))(x1)
  x1 = Conv2D(256, (3, 3), activation='relu')(x1)
  x1 = MaxPooling2D((2, 2))(x1)
  x1 = Flatten()(x1)

  x2 = Conv2D(64, (3, 3), activation='relu')(depth_input_2)
  x2 = MaxPooling2D((2, 2))(x2)
  x2 = Conv2D(128, (3, 3), activation='relu')(x2)
  x2 = MaxPooling2D((2, 2))(x2)
  x2 = Conv2D(256, (3, 3), activation='relu')(x2)
  x2 = MaxPooling2D((2, 2))(x2)
  x2 = Flatten()(x2)

  # Concatene as saídas das camadas convolucionais
  concatenated_output = Concatenate()([x1, x2])

  x = Dense(256, activation='relu')(concatenated_output)

  # Saída
  output = Dense(len(campos), activation='linear')(x)

  # Construa o modelo com as duas entradas
  model = Model(inputs=[depth_input_1, depth_input_2], outputs=output)

  # Compile o modelo
  model.compile(optimizer='adam', loss='mean_squared_error')
  return model

inference_model = create_model()
inference_model.load_weights("estimativa_de_peso_vivo_03.keras")

model = load_model(weights_path, device=device)
model_seg = YOLO('yolov8s-seg.pt')

# Carregando o CSV com os dados
data = pd.read_csv('dataset.csv')

# Dicionário de tradução das saídas
traducao = {
    'live weithg': 'Peso Vivo',
    'withers height': 'Altura da Cernelha',
    'hip height': 'Altura do Quadril',
    'chest depth': 'Profundidade do Peito',
    'chest width': 'Largura do Peito',
    'ilium width': 'Largura do Ílio',
    'hip joint width': 'Largura da Articulação do Quadril',
    'oblique body length': 'Comprimento do Corpo Oblíquo',
    'hip length': 'Comprimento do Quadril',
    'heart girth': 'Circunferência Torácica'
}


def get_depth(image):
    depths = []
    i = 1
    cv2.imwrite('/content/dust3r/temp_img/1.png',image)
    imgs = load_images('/content/dust3r/temp_img', size=512)
    if len(imgs) == 1:
        imgs = [imgs[0], copy.deepcopy(imgs[0])]
        imgs[1]['idx'] = 1

    scenegraph_type = f"swin-1"

    pairs = make_pairs(imgs, scene_graph=scenegraph_type, prefilter=None, symmetrize=True)
    output = inference(pairs, model, device)

    mode = GlobalAlignerMode.PointCloudOptimizer if len(imgs) > 2 else GlobalAlignerMode.PairViewer
    scene = global_aligner(output, device=device, mode=mode)

    if mode == GlobalAlignerMode.PointCloudOptimizer:
        loss = scene.compute_global_alignment(init='mst')

    pts3d = to_numpy(scene.get_pts3d())

    for pts in pts3d:
      normalizedpts3d = cv2.normalize(pts,  pts, 0, 255, cv2.NORM_MINMAX)
      one_channel_3d = cv2.cvtColor(normalizedpts3d, cv2.COLOR_BGR2GRAY)
      depths.append(one_channel_3d)
      print(f"Imagem convertida {i}")
      i += 1
    return depths

def get_mask(image):
  results = model_seg.predict(image.copy(), conf=0.40)
  masks = results[0].masks
  mask1 = masks.masks
  mask = mask1.data.cpu().numpy()
  mask = mask.squeeze()
  return mask

st.title("ENGORD.AI - BIOMETRIA ")
uploaded_file = st.file_uploader("Faça o upload da imagem:", type=['jpg', 'png'])

if uploaded_file is not None:
  dim = 256
  file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
  image = cv2.imdecode(file_bytes, 1)


  with st.spinner('Realizando predições'):
    col1, col2, col3, col4, col5 = st.columns(5)
    col1.image(image, channels="BGR")

    depths = get_depth(image)
    depth = depths[0]
    fig, ax = plt.subplots()
    ax.imshow(depth)
    col2.pyplot(fig)

    mask = get_mask(image)
    fig2, ax2 = plt.subplots()
    ax2.imshow(mask)
    mask = cv2.resize(mask,(dim,dim))

    col2.pyplot(fig2)

    resize_depth = cv2.resize(depth,(dim,dim))
    norm_depth = resize_depth / 255.

    depth_fim = np.expand_dims(norm_depth, axis=0)
    mask_fim = np.expand_dims(mask, axis=0)
    prediction = inference_model.predict([depth_fim, mask_fim])

    col3.subheader('Estimativa')
    predictions = []
    try:
        for i, campo in enumerate(traducao.keys()):
            col3.markdown(f'<b>{traducao[campo]}</b>: {round(float(prediction[0][i]),2)}', unsafe_allow_html=True)
            predictions.append(prediction[0][i])

    except:
        pass
    col4.subheader('Dado Real')
    col5.subheader('Precisão')
    N = int(uploaded_file.name.split(".")[0])-1
    precisao_total = []
    for i,campo in enumerate(traducao.keys()):
        if campo == "N" or 'time' in campo or 'Data' in campo:
            continue
        col4.markdown(f'<b>{traducao[campo]}</b>: {data.iloc[N][campo]}', unsafe_allow_html=True)
        v = float(data.iloc[N][campo])
        p = predictions[i]
        per = v/ 100
        diff = abs(v-p)
        percent = 100 - (diff/per)
        precisao_total.append(percent)
        col5.markdown(f'{round(percent,2)}%', unsafe_allow_html=True)
    precisao_modelo = sum(precisao_total)/len(precisao_total)
    col4.subheader(f"Precisão do Modelo: {round(precisao_modelo,2)}%")

In [ ]:
%cd /content/dust3r

In [ ]:
!ls

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8504

## **DEMO DUST3R**

In [ ]:
%cd /content/dust3r/

In [ ]:
!python3 demo.py --weights /content/dust3r/checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth

... loading model from /content/dust3r/checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>
Outputing stuff in /tmp/tmpoxgsdxf4dust3r_gradio_demo
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://6a8f9627340a3881cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/py